In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
from IPython.display import display
import random
pd.options.display.max_columns = None

test_obs = 401590
train_obs = 1708338

In [3]:
%%time
from ast import literal_eval

def parse(csv_path='/Volumes/My Passport/Kaggle Competition/train_sample.csv', nrows=None):

    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    df['hits']=df['hits'].apply(literal_eval)
    df['hits']=df['hits'].str[0]
    df=df[pd.notnull(df['hits'])]
    
    df['customDimensions']=df['customDimensions'].apply(literal_eval)
    df['customDimensions']=df['customDimensions'].str[0]
    df=df[pd.notnull(df['customDimensions'])]
    
    
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource','hits','customDimensions']
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    return df
    
def load_data(nrows=None):
    train_df = parse(nrows=nrows)
    #test_df = parse("/Volumes/My Passport/Kaggle Competition/train_v2.csv",nrows)
    return train_df#,test_df

df_train=load_data(nrows=10000)

CPU times: user 39.3 s, sys: 1.22 s, total: 40.5 s
Wall time: 42.6 s


In [5]:
df_train.shape

(2532, 116)

In [6]:
# make NANs as 0 and set all values as float
#df_train['totals.transactionRevenue'] = df_train['totals.transactionRevenue'].astype(float)
columns_to_float = ['visitNumber', 'totals.bounces', 'totals.sessionQualityDim', 'totals.hits', 'totals.newVisits', 'totals.pageviews','totals.timeOnSite','totals.totalTransactionRevenue', 'totals.transactionRevenue','totals.transactions','totals.visits']
for column in columns_to_float:
    df_train[column].fillna(0, inplace=True)
    df_train[column] = df_train[column].astype(float)

In [7]:
#df_train['bought_something'] = df_train['totals.transactionRevenue'] > 0
df_train['bought_something'] = df_train['totals.transactionRevenue'].apply(lambda x: 1 if x > 0 else 0)
df_train['country_us'] = df_train['geoNetwork.country'].apply(lambda x: 1 if x == 'United States' else 0)

In [8]:
# Gather date/time data
from datetime import datetime
format_str = '%Y%m%d'
df_train['formatted_date'] = df_train['date'].apply(lambda x: datetime.strptime(str(x), format_str))
df_train['month'] = df_train['formatted_date'].apply(lambda x:x.month)
df_train['weekday'] = df_train['formatted_date'].apply(lambda x:x.weekday()) #mon to sun; 0 to 6
df_train['visitStartHour'] = df_train['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))
df_train.drop(['date','visitStartTime'], axis=1, inplace=True)


In [9]:
# group sources
to_replace = [r'.*google.*',r'.*yahoo.*',r'.*\.facebook.*',r'(\.com)',r'(ml\.)',r'(m\.)',] #to group all google ,facebook types
value = [r'google',r'yahoo',r'facebook',r'',r'',r'']
df_train['trafficSource.source'] = df_train['trafficSource.source'].replace(to_replace, value, regex=True)

In [57]:
df_train = df_train.drop(['revenue_prediction'],axis=1)

In [58]:
pd.to_csv(df_train, 'df_eda.csv')

AttributeError: module 'pandas' has no attribute 'write_csv'

In [10]:
train_columns_to_keep = [
    'visitNumber',
    'device.isMobile',
    'totals.bounces',
    'totals.hits',
    'totals.newVisits',
    'totals.pageviews',
    'totals.sessionQualityDim',
    'totals.timeOnSite',
    'totals.visits',
    'trafficSource.medium',
    'country_us',
    'month',
    'totals.transactions',
    'totals.transactionRevenue',
    'totals.totalTransactionRevenue',
    'bought_something'
]
df_train_sub = df_train.filter(train_columns_to_keep , axis = 1)
df_train_sub.head()

,visitNumber,device.isMobile,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.visits,trafficSource.medium,country_us,month,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,bought_something
0,1.0,False,0.0,2.0,1.0,2.0,2.0,12.0,1.0,(none),1,10,0.0,0.0,0.0,0
1,1.0,False,0.0,2.0,1.0,2.0,1.0,9.0,1.0,referral,0,10,0.0,0.0,0.0,0
3,1.0,True,0.0,2.0,0.0,2.0,1.0,32.0,1.0,organic,0,10,0.0,0.0,0.0,0
4,2.0,False,0.0,2.0,1.0,2.0,1.0,1266.0,1.0,organic,0,10,0.0,0.0,0.0,0
5,1.0,False,0.0,2.0,1.0,2.0,1.0,11.0,1.0,organic,0,10,0.0,0.0,0.0,0


In [11]:
columns_to_dummies = [
    'trafficSource.medium',
]
df_train_dum = pd.get_dummies(df_train_sub, columns = columns_to_dummies, drop_first=True)
df_train_sub = df_train_dum.drop(['totals.transactions','totals.transactionRevenue','totals.totalTransactionRevenue','bought_something'],axis=1)
df_train_response = df_train_dum.filter(['totals.transactions','totals.transactionRevenue','totals.totalTransactionRevenue','bought_something'])

In [12]:
bought_sub = df_train_dum[df_train_dum['bought_something'] == 1]
bought_sub_response = bought_sub.filter(['totals.transactions','totals.transactionRevenue','totals.totalTransactionRevenue','bought_something'])
bought_sub_regress = bought_sub.drop(['totals.transactions','totals.transactionRevenue','totals.totalTransactionRevenue','bought_something'],axis=1)

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(max_depth=10, random_state=0)
fit = clf.fit(df_train_sub, df_train_response['bought_something'])
bought_predictions = fit.predict(df_train_sub)
df_train_sub['bought_predictions'] = bought_predictions
df_train_dum['bought_predictions'] = bought_predictions
df_train_response['bought_predictions'] = bought_predictions

In [27]:
# approx the same amount predicted bought something as actually did
print(sum(df_train_dum['bought_something']))
print(sum(df_train_sub['bought_predictions']))

19
16


In [34]:
df_train_dum[df_train_dum['bought_something'] == 1]

,visitNumber,device.isMobile,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.visits,country_us,month,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,bought_something,trafficSource.medium_affiliate,trafficSource.medium_cpc,trafficSource.medium_cpm,trafficSource.medium_organic,trafficSource.medium_referral,bought_predictions
109,1.0,False,0.0,9.0,0.0,9.0,13.0,261.0,1.0,1,10,1.0,15190000.0,1.719000e+07,1,0,0,0,1,0,1
171,1.0,False,0.0,23.0,0.0,21.0,68.0,1312.0,1.0,1,10,1.0,44000000.0,5.100000e+07,1,0,0,0,0,0,0
190,1.0,False,0.0,138.0,0.0,111.0,71.0,4459.0,1.0,0,10,1.0,900430000.0,9.064300e+08,1,0,0,0,1,0,1
1095,2.0,False,0.0,13.0,0.0,11.0,0.0,171.0,1.0,1,9,1.0,68030000.0,7.303000e+07,1,0,0,0,0,0,1
1096,2.0,False,0.0,13.0,0.0,12.0,0.0,479.0,1.0,1,9,1.0,26250000.0,2.925000e+07,1,0,0,0,0,0,1
1115,2.0,False,0.0,17.0,0.0,17.0,0.0,424.0,1.0,1,9,1.0,8380000.0,1.238000e+07,1,0,0,0,0,0,1
1136,1.0,False,0.0,24.0,0.0,21.0,0.0,571.0,1.0,1,9,2.0,546320000.0,2.103690e+09,1,0,0,0,0,0,1
1157,1.0,False,0.0,47.0,0.0,35.0,0.0,3089.0,1.0,1,9,1.0,5150000.0,6.150000e+06,1,0,0,0,0,0,0
1163,2.0,False,0.0,72.0,0.0,50.0,0.0,1172.0,1.0,1,9,2.0,338970000.0,6.729600e+08,1,0,0,0,0,0,1
1447,1.0,False,0.0,10.0,0.0,10.0,78.0,146.0,1.0,1,9,1.0,207950000.0,2.119500e+08,1,0,0,0,0,0,1


In [38]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(max_depth=10, random_state=0)
tree_fit = clf.fit(bought_sub_regress, bought_sub_response['totals.transactionRevenue'])
#lin_fit = LinearRegression().fit(bought_sub_regress, bought_sub_response['totals.transactionRevenue'])

In [40]:
list_of_values = []
for index, row in df_train_sub.iterrows():
    if row['bought_predictions'] == 1:
        list_of_values.append(tree_fit.predict([row[:-1]])[0])
    else:
        list_of_values.append(0)


In [44]:
df_train_dum['regression_predictions'] = list_of_values

In [48]:
df_train_dum[df_train_dum['totals.transactionRevenue'] == 0]

,visitNumber,device.isMobile,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.visits,country_us,month,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,bought_something,trafficSource.medium_affiliate,trafficSource.medium_cpc,trafficSource.medium_cpm,trafficSource.medium_organic,trafficSource.medium_referral,bought_predictions,regression_predictions
0,1.0,False,0.0,2.0,1.0,2.0,2.0,12.0,1.0,1,10,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0
1,1.0,False,0.0,2.0,1.0,2.0,1.0,9.0,1.0,0,10,0.0,0.0,0.0,0,0,0,0,0,1,0,0.0
3,1.0,True,0.0,2.0,0.0,2.0,1.0,32.0,1.0,0,10,0.0,0.0,0.0,0,0,0,0,1,0,0,0.0
4,2.0,False,0.0,2.0,1.0,2.0,1.0,1266.0,1.0,0,10,0.0,0.0,0.0,0,0,0,0,1,0,0,0.0
5,1.0,False,0.0,2.0,1.0,2.0,1.0,11.0,1.0,0,10,0.0,0.0,0.0,0,0,0,0,1,0,0,0.0
6,1.0,False,0.0,2.0,1.0,2.0,1.0,148.0,1.0,0,10,0.0,0.0,0.0,0,0,0,0,1,0,0,0.0
7,1.0,True,0.0,2.0,0.0,2.0,1.0,17.0,1.0,0,10,0.0,0.0,0.0,0,0,0,0,1,0,0,0.0
8,3.0,False,0.0,2.0,1.0,2.0,1.0,30.0,1.0,0,10,0.0,0.0,0.0,0,0,0,0,1,0,0,0.0
9,1.0,False,0.0,2.0,1.0,2.0,1.0,8.0,1.0,1,10,0.0,0.0,0.0,0,0,0,0,1,0,0,0.0
10,1.0,True,0.0,3.0,1.0,3.0,1.0,47.0,1.0,1,10,0.0,0.0,0.0,0,0,1,0,0,0,0,0.0


In [52]:
df_train['revenue_prediction'] = list_of_values

In [54]:
df_train[df_train['totals.transactionRevenue'] > 0]

,Unnamed: 0,channelGrouping,fullVisitorId,socialEngagementType,visitId,visitNumber,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,hits.appInfo.exitScreenName,hits.appInfo.landingScreenName,hits.appInfo.screenDepth,hits.appInfo.screenName,hits.contentGroup.contentGroup1,hits.contentGroup.contentGroup2,hits.contentGroup.contentGroup3,hits.contentGroup.contentGroup4,hits.contentGroup.contentGroup5,hits.contentGroup.contentGroupUniqueViews1,hits.contentGroup.contentGroupUniqueViews2,hits.contentGroup.contentGroupUniqueViews3,hits.contentGroup.previousContentGroup1,hits.contentGroup.previousContentGroup2,hits.contentGroup.previousContentGroup3,hits.contentGroup.previousContentGroup4,hits.contentGroup.previousContentGroup5,hits.customDimensions,hits.customMetrics,hits.customVariables,hits.dataSource,hits.eCommerceAction.action_type,hits.eCommerceAction.step,hits.eventInfo.eventAction,hits.eventInfo.eventCategory,hits.eventInfo.eventLabel,hits.exceptionInfo.isFatal,hits.experiment,hits.hitNumber,hits.hour,hits.isEntrance,hits.isExit,hits.isInteraction,hits.item.currencyCode,hits.minute,hits.page.hostname,hits.page.pagePath,hits.page.pagePathLevel1,hits.page.pagePathLevel2,hits.page.pagePathLevel3,hits.page.pagePathLevel4,hits.page.pageTitle,hits.page.searchCategory,hits.page.searchKeyword,hits.product,hits.promotion,hits.promotionActionInfo.promoIsClick,hits.promotionActionInfo.promoIsView,hits.publisher_infos,hits.referer,hits.social.hasSocialSourceReferral,hits.social.socialInteractionNetworkAction,hits.social.socialNetwork,hits.time,hits.transaction.currencyCode,hits.type,customDimensions.index,customDimensions.value,bought_something,country_us,formatted_date,month,weekday,visitStartHour,revenue_prediction
109,109,Referral,1188936504674178471,Not Socially Engaged,1508160410,1.0,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,New York,not available in demo dataset,Americas,United States,not available in demo dataset,not available in demo dataset,New York NY,(not set),not available in demo dataset,New York,Northern America,0.0,9.0,0.0,9.0,13.0,261.0,1.719000e+07,15190000.0,1.0,1.0,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google,shop.googlemerchandisestore.com/myaccount.html...,shop.googlemerchandisestore.com/google+redesig...,0,shop.googlemerchandisestore.com/google+redesig...,(not set),Apparel,Womens,(not set),(not set),NaN,1,1,(entrance),(entrance),(entrance),(entrance),(entrance),[],[],[],web,0,1,NaN,NaN,NaN,True,